# English and Spanish used cars sales description using SK<br/>Text generation scenario

Learning objectives:

- What does SK provide for me?
  - A configurable Kernel with pluggable architecture
  - The models to work with the OpenAI endpoints
  - A resilient HttpClient that can handle disconnections and throttlihg
  - SK functions that can be pipe to achieve complex orchestrations such a text generation and transaltion
  - Ability to handle prompts and completions

## Load the required .NET packages and supporting classes

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.4.0"

#r "nuget: dotenv.net, 3.1.2"

using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using dotenv.net;

#!import Models/Models.cs

// Json Options
readonly JsonSerializerOptions s_jsonOptions = new() { WriteIndented = true };

Installed Packages dotenv.net, 3.1.2 Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Core, 1.4.0

## Load the OpenAI endpoint and API key from environment variables or an .env file

In [2]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at: https://alemorfr.openai.azure.com/ with key 32c87...


## Mock a database call to get a list of car

In [3]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };
  
List<Car> MockDBCall() {  
  return mockCarData;
}

## Function to get a short car description from the car properties

In [4]:
// Reminder: I created this function, to make more impact, but the car record has a ToString defition to return the same information
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Create an instance of a kernel

- Here notice how elegantly the kernel is configured

In [5]:
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .Build();

## Create the sales definition and translation in-line functions and add them to the kernel

In [6]:
// Define a function to get a used car sales description
const string fnSalesDefinition = @"Get a used car sales description for the following: 
{{$input}}.";

var salesDescriptionFunc = kernel.CreateFunctionFromPrompt(fnSalesDefinition, 
    new OpenAIPromptExecutionSettings() { MaxTokens = 200, Temperature = 0.5, TopP = 1 });

// Define a function to translate English to Spanish
const string fnESTranslatorDefinition = @"Translate the following English text to Spanish: 
{{$input}}.";

var esTranslatorFunc = kernel.CreateFunctionFromPrompt(fnESTranslatorDefinition, 
    new OpenAIPromptExecutionSettings() { MaxTokens = 200, Temperature = 0.5, TopP = 1 });

## Using the kernel and sk functions, get a car sales description and translate it into Spanish

In [7]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();
var carDesc = GetCarDescription(cars[0]);
cars[0].ToString()

2015 Ford Explorer Silver V6 Platinum with 10000 miles for $16500

## Get a sales description and a translation for all the cars in the mock database

In [8]:
// Keep a list of processed cars
var processedCars = new List<CarTranslation>();

foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing:\n{desc}");
  
  // Use the SK sales description function to get a sales for a car
  var arguments = new KernelArguments()
        {
            ["input"] = desc
        };
  var result = await kernel.InvokeAsync(salesDescriptionFunc, arguments);
  Console.WriteLine($"Sales Description:\n{result}");

  // Get the usage for the call
  //Console.WriteLine(JsonSerializer.Serialize(result.FunctionResults.LastOrDefault()?.GetOpenAIChatResult()?.Usage, s_jsonOptions));  
  
  // Use the SK translation function to translate the sales description into Spanish
  arguments.Clear();
  arguments["input"] = result.ToString();
  var esTranslation = await kernel.InvokeAsync(esTranslatorFunc,arguments);
  Console.WriteLine($"Translation:\n{esTranslation}\n");
  
  // Add processed the EN and ES sales description
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), esTranslation.ToString()));
}

// Show the processed cars
processedCars

Processing:
2015 Ford Explorer Silver V6 Platinum with 10000 miles for $16500
Sales Description:
Looking for a reliable, spacious, and luxurious SUV? Look no further than this 2015 Ford Explorer Platinum! With only 10,000 miles on the odometer, this vehicle is practically brand new. The sleek silver exterior is sure to turn heads, and the powerful V6 engine provides a smooth and comfortable ride. 

Inside, you'll find a spacious and comfortable cabin with plenty of room for passengers and cargo. The Platinum trim level comes loaded with features, including leather seats, a premium sound system, a touchscreen infotainment system, and much more. 

Best of all, this 2015 Ford Explorer Platinum is priced to sell at just $16,500. That's a great value for a vehicle that's in such excellent condition and loaded with so many features. Don't miss your chance to own this amazing SUV!
Translation:
¿Buscas un SUV confiable, espacioso y lujoso? ¡No busques más que este Ford Explorer Platinum 2015! 

index value 0 CarTranslation { VIN = 123, En = Looking for a reliable, spacious, and luxurious SUV? Look no further than this 2015 Ford Explorer Platinum! With only 10,000 miles on the odometer, this vehicle is practically brand new. The sleek silver exterior is sure to turn heads, and the powerful V6 engine prov... VIN 123 En Looking for a reliable, spacious, and luxurious SUV? Look no further than this 2015 Ford Explorer Platinum! With only 10,000 miles on the odometer, this vehicle is practically brand new. The sleek silver exterior is sure to turn heads, and the powerful V6 engine provides a smooth and comfortable ride. 

Inside, you'll find a spacious and comfortable cabin with plenty of room for passengers and cargo. The Platinum trim level comes loaded with features, including leather seats, a premium sound system, a touchscreen infotainment system, and much more. 

Best of all, this 2015 Ford Explorer Platinum is priced to sell at just $16,500. That's a great value for a vehicle that's in such excellent condition and loaded with so many features. Don't miss your chance to own this amazing SUV! ES ¿Buscas un SUV confiable, espacioso y lujoso? ¡No busques más que este Ford Explorer Platinum 2015! Con solo 10,000 millas en el odómetro, este vehículo es prácticamente nuevo. El elegante exterior plateado seguramente llamará la atención, y el potente motor V6 proporciona un viaje suave y cómodo.

En el interior, encontrarás una cabina espaciosa y cómoda con mucho espacio para pasajeros y carga. El nivel de equipamiento Platinum viene cargado con características, que incluyen asientos de cuero, un sistema de sonido premium, un sistema de infoentretenimiento con pantalla táctil y mucho más.

Lo mejor de todo es que este Ford Explorer Platinum 2015 está a la venta por solo $16,500. Ese es un gran valor para un vehículo que está en excelentes condiciones y está cargado 1 CarTranslation { VIN = 456, En = Looking for a powerful and stylish sports car that will turn heads wherever you go? Check out this stunning 2018 Ford Mustang V8 Sports in a gorgeous blue color! With only 10,000 miles on the odometer, this car is practically brand new and ready to hit the road.\n\nU... VIN 456 En Looking for a powerful and stylish sports car that will turn heads wherever you go? Check out this stunning 2018 Ford Mustang V8 Sports in a gorgeous blue color! With only 10,000 miles on the odometer, this car is practically brand new and ready to hit the road.

Under the hood, you'll find a powerful V8 engine that delivers an exhilarating driving experience. Whether you're cruising down the highway or taking on tight corners, this car is built to perform. And with its sleek and aerodynamic design, you'll feel like you're driving a true sports car.

Inside, you'll find a comfortable and stylish interior with all the latest features and technology. From the infotainment system to the premium sound system, this car has everything you need to stay connected and entertained on the road.

So why wait? Come check out this amazing 2018 Ford Mustang V8 Sports today and experience the thrill of driving a true sports car. At only $250,000 ES ¿Buscas un automóvil deportivo potente y elegante que llame la atención dondequiera que vayas? ¡Echa un vistazo a este impresionante Ford Mustang V8 Sports 2018 en un hermoso color azul! Con solo 10,000 millas en el odómetro, este automóvil es prácticamente nuevo y está listo para salir a la carretera.

Debajo del capó, encontrarás un potente motor V8 que ofrece una experiencia de conducción emocionante. Ya sea que estés cruzando la autopista o enfrentando curvas cerradas, este automóvil está construido para rendir. Y con su diseño elegante y aerodinámico, sentirás que estás conduciendo un verdadero automóvil deportivo.

En el interior, encontrarás un interior cómodo y elegante con todas las últimas características y tecnología. Desde el 2 CarTranslation { VIN = 789, En = Looking for a powerful and stylish SUV that won't bre